In [2]:
#This is an example notebook that shows how the Event_Visualizer class can be used.
%matplotlib auto
%load_ext autoreload
%autoreload 2
from event_visualizer import Event_Visualizer
import ROOT as r

Using matplotlib backend: TkAgg
Welcome to JupyROOT 6.26/06


In [8]:
EV = Event_Visualizer()

input_file = r.TFile("~/pioneer/main/run/test.root")
EV.visualize_event(input_file, display_text_output=False);

Event #  0
Event #  1
Event #  2


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Event #  3


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Event #  4


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Event #  5
Event #  6
Event #  7
Event #  8


In [ ]:
#TESTING STUFF

hist = r.TH1F("h1", "Edep", 100, 0, 100)

# for entry in tree:
#     for atar in entry.atar:
#         #hist.Fill(calo.GetTotalEnergyDeposit())
#         print(atar.edep)

canv = r.TCanvas()
hist.Draw()
canv.Update()
canv.Draw()